In [3]:
#imports
import logging
import os
import sys
import shutil
import tempfile
from monai.data import Dataset

import matplotlib.pyplot as plt
import torch
import torch.nn as nn
#from torch.utils.tensorboard import SummaryWriter
import numpy as np

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    Resize,
    ScaleIntensity,
)

import glob
import nibabel as nib
  



RuntimeError: Only a single TORCH_LIBRARY can be used to register the namespace triton; please put all of your definitions in a single TORCH_LIBRARY block.  If you were trying to specify implementations, consider using TORCH_LIBRARY_IMPL (which can be duplicated).  If you really intended to define operators for a single namespace in a distributed way, you can use TORCH_LIBRARY_FRAGMENT to explicitly indicate this.  Previous registration of TORCH_LIBRARY was registered at /dev/null:2623; latest registration was registered at /dev/null:2623

# Training

### Loading the data

In [5]:
pin_memory = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
data_dir = "L:/Basic/divi/jstoker/slicer_pdac/Master Students WS 24/Martijn/data/Training/paired_scans" #fill in training datapath
nifti_images = sorted(glob.glob(os.path.join(data_dir, "*.nii.gz")))      

In [ ]:
class PairedMedicalDataset(Dataset):
    def __init__(self, image_pairs, metadata, labels, transform=None):
        self.image_pairs = image_pairs
        self.metadata = metadata
        self.labels = labels
        self.transform = transform
    
    def __len__(self):
        return len(self.image_pairs)
    
    def __getitem__(self, idx):
        img1_path, img2_path = self.image_pairs[idx]
        
        # Load images using nibabel (for NIfTI)
        img1 = nib.load(img1_path).get_fdata()
        img2 = nib.load(img2_path).get_fdata()

        # Add channel dimension for CNN input (C, H, W, D)
        img1 = np.expand_dims(img1, axis=0)
        img2 = np.expand_dims(img2, axis=0)
        
        metadata = self.metadata[idx]
        label = self.labels[idx]
        
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)

        # Convert to tensor
        img1 = torch.tensor(img1, dtype=torch.float32)
        img2 = torch.tensor(img2, dtype=torch.float32)
        label = torch.tensor(label, dtype=torch.float32)
        
        return img1, img2, metadata, label

In [19]:
# Create pairs (e.g., first and second file are paired)
image_pairs = [(nifti_images[i], nifti_images[i + 1]) for i in range(0, len(nifti_images) - 1, 2)]
labels = None #Fill in correct path. response, PFS, and OS

# Create dataset        Change resize!!!!!!!!!!!!!!
train_dataset = PairedMedicalDataset(image_pairs, labels, transform=[ScaleIntensity(), EnsureChannelFirst(), Resize((96, 96, 96))])

# DataLoader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)

### Initialize model

In [ ]:
class SiameseNetwork(nn.Module):
    def __init__(self, base_model):
        super(SiameseNetwork, self).__init__()
        self.base_model = base_model
        self.classifier = nn.Sequential(
            nn.Linear(),
            nn.ReLU(),
            nn.Linear(,8),      #8 classes (Path. resp. 2, PFS 3, OS 3)
            nn.Sigmoid()
        )

    
    def forward(self, image1, image2, metadata):
        # Pass both inputs through the shared model
        output1 = self.base_model(image1)
        output2 = self.base_model(image2)
        #flatten image output
        combined_embeddings = torch.cat((output1, output2, metadata), dim=1)
        output3 = self.classifier(combined_embeddings)
        return output3


In [20]:
import monai.bundle
print(monai.bundle.get_all_bundles_list())

[('spleen_ct_segmentation', '0.5.9'), ('pancreas_ct_dints_segmentation', '0.5.0'), ('brats_mri_segmentation', '0.5.3'), ('spleen_deepedit_annotation', '0.5.6'), ('swin_unetr_btcv_segmentation', '0.5.6'), ('ventricular_short_axis_3label', '0.3.3'), ('mednist_gan', '0.4.3'), ('pathology_tumor_detection', '0.6.2'), ('renalStructures_UNEST_segmentation', '0.2.5'), ('wholeBrainSeg_Large_UNEST_segmentation', '0.2.5'), ('prostate_mri_anatomy', '0.3.4'), ('valve_landmarks', '0.5.0'), ('lung_nodule_ct_detection', '0.6.9'), ('endoscopic_tool_segmentation', '0.6.1'), ('endoscopic_inbody_classification', '0.5.0'), ('breast_density_classification', '0.1.7'), ('mednist_reg', '0.0.6'), ('pathology_nuclei_classification', '0.2.0'), ('pathology_nuclick_annotation', '0.2.1'), ('pathology_nuclei_segmentation_classification', '0.2.6'), ('wholeBody_ct_segmentation', '0.2.5'), ('brats_mri_generative_diffusion', '1.1.3'), ('brats_mri_axial_slices_generative_diffusion', '1.1.3'), ('renalStructures_CECT_segmen

In [24]:
import monai
import tqdm
from monai.networks.nets import SwinUNETR

pretrained_model = monai.bundle.load(
    name="swin_unetr_btcv_segmentation", bundle_dir="C:/Users/P095550/OneDrive - Amsterdam UMC/Documenten/GitHub/CRLM-morph-features/monai_whole_body_ct_model/monai_wholebody_ct_segmentation_0.2.5"
)


# Define the network architecture
network = SwinUNETR(
    spatial_dims=3,
    in_channels=1,
    out_channels=14,
    img_size=(96, 96, 96),
    feature_size=48,
    use_checkpoint=True
    )

# Load the pretrained weights into the network
network.load_state_dict(pretrained_model)






c:\Users\P095550\OneDrive - Amsterdam UMC\Documenten\GitHub\CRLM-morph-features\.venv\lib\site-packages\monai\utils\deprecate_utils.py:221: FutureWarning: monai.networks.nets.swin_unetr SwinUNETR.__init__:img_size: Argument `img_size` has been deprecated since version 1.3. It will be removed in version 1.5. The img_size argument is not required anymore and checks on the input size are run during forward().
  warn_deprecated(argname, msg, warning_category)


<All keys matched successfully>

In [26]:
encoder = network.swinViT
print(encoder)

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv3d(1, 48, kernel_size=(2, 2, 2), stride=(2, 2, 2))
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (layers1): ModuleList(
    (0): BasicLayer(
      (blocks): ModuleList(
        (0-1): 2 x SwinTransformerBlock(
          (norm1): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=48, out_features=144, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=48, out_features=48, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
          (mlp): MLPBlock(
            (linear1): Linear(in_features=48, out_features=192, bias=True)
            (linear2): Linear(in_features=192, out_features=48, bias=True)
            (fn): GELU(approxim

In [1]:
import torch

# Example input tensor
input_tensor = torch.randn(1, 1, 96, 96, 96)  # Batch size 1, 1 channel, 96x96x96 image

# Pass the input through the encoder
features = encoder(input_tensor)

# Print the shape of the extracted features
print(f"Extracted features shape: {features.shape}")

KeyboardInterrupt: 

In [22]:
print(pretrained_model.keys())

odict_keys(['dints_space.cell_tree.(0, 0).op.ops.0.conv.weight', 'dints_space.cell_tree.(0, 0).op.ops.0.conv_1.weight', 'dints_space.cell_tree.(0, 0).op.ops.0.norm.weight', 'dints_space.cell_tree.(0, 0).op.ops.0.norm.bias', 'dints_space.cell_tree.(0, 1).preprocess.conv.weight', 'dints_space.cell_tree.(0, 1).preprocess.norm.weight', 'dints_space.cell_tree.(0, 1).preprocess.norm.bias', 'dints_space.cell_tree.(0, 1).op.ops.0.conv.weight', 'dints_space.cell_tree.(0, 1).op.ops.0.conv_1.weight', 'dints_space.cell_tree.(0, 1).op.ops.0.norm.weight', 'dints_space.cell_tree.(0, 1).op.ops.0.norm.bias', 'dints_space.cell_tree.(0, 2).preprocess.conv_1.weight', 'dints_space.cell_tree.(0, 2).preprocess.conv_2.weight', 'dints_space.cell_tree.(0, 2).preprocess.norm.weight', 'dints_space.cell_tree.(0, 2).preprocess.norm.bias', 'dints_space.cell_tree.(0, 2).op.ops.0.conv.weight', 'dints_space.cell_tree.(0, 2).op.ops.0.conv_1.weight', 'dints_space.cell_tree.(0, 2).op.ops.0.norm.weight', 'dints_space.cell_

In [ ]:
from monai.networks.nets import SegResNet
map_location=torch.device('cpu')

# Step 2: Load the saved state dictionary
model_path = "C:/Users/P095550/OneDrive - Amsterdam UMC/Documenten/GitHub/CRLM-morph-features/monai_whole_body_ct_model/monai_wholebody_ct_segmentation_0.2.5/_/models/model.pt"

state_dict = torch.load(model_path)

network = SegResNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=105,
    init_filters=32,
    blocks_down=[1, 2, 2, 4],
    blocks_up=[1, 1, 1],
    dropout_prob=0.2
)
# Step 3: Load the weights into the model
network.load_state_dict(state_dict)



RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [ ]:
from monai.networks.nets import SegResNet
"""
"network_def": {
        "_target_": "SegResNet",
        "spatial_dims": 3,
        "in_channels": 1,
        "out_channels": 105,
        "init_filters": 32,
        "blocks_down": [
            1,
            2,
            2,
            4
        ],
        "blocks_up": [
            1,
            1,
            1
        ],
        "dropout_prob": 0.2
    }
"""


network = SegResNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=105,
    init_filters=32,
    blocks_down=[1, 2, 2, 4],
    blocks_up=[1, 1, 1],
    dropout_prob=0.2
)
network.load_statedict(pretrained_model)

### Training